# MHW Event 3D

In [26]:
# imports
import numpy as np
import os
from matplotlib import pyplot as plt

from datetime import date

import pandas
import sqlalchemy
from mayavi import mlab

# Load

In [2]:
mhw_file = '/home/xavier/Projects/Oceanography/MHWs/db/mhws_allsky_defaults.db'
tst_file = '/home/xavier/Projects/Oceanography/MHWs/db/test_mhws_allsky.db'
#
engine = sqlalchemy.create_engine('sqlite:///'+mhw_file)
connection = engine.connect()
connection

In [3]:
mhw_events = pandas.read_sql_table('MHW_Events', con=engine, 
                               columns=['date', 'lon', 'lat', 'duration' , 
                                        'ievent', 'time_peak', 'time_start'])
mhw_events.head()

,date,lon,lat,duration,ievent,time_peak,time_start
0,1982-01-01,0.125,-71.375,13,0,723552,723546
1,1982-12-20,0.125,-71.375,33,1,723901,723899
2,1985-01-30,0.125,-71.375,10,2,724677,724671
3,1988-01-10,0.125,-71.375,9,3,725752,725746
4,1991-01-09,0.125,-71.375,52,4,726858,726841


In [4]:
mhw_events = mhw_events.set_index('date')
mhw_events.head()

,lon,lat,duration,ievent,time_peak,time_start
date,,,,,,
1982-01-01,0.125,-71.375,13,0,723552,723546
1982-12-20,0.125,-71.375,33,1,723901,723899
1985-01-30,0.125,-71.375,10,2,724677,724671
1988-01-10,0.125,-71.375,9,3,725752,725746
1991-01-09,0.125,-71.375,52,4,726858,726841


## Add start date

In [5]:
mhw_events['peak_date'] = pandas.to_datetime([date.fromordinal(tt) for tt in mhw_events['time_peak']])

In [6]:
mhw_events.head()

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
1982-01-01,0.125,-71.375,13,0,723552,723546,1982-01-07
1982-12-20,0.125,-71.375,33,1,723901,723899,1982-12-22
1985-01-30,0.125,-71.375,10,2,724677,724671,1985-02-05
1988-01-10,0.125,-71.375,9,3,725752,725746,1988-01-16
1991-01-09,0.125,-71.375,52,4,726858,726841,1991-01-26


# Pick one

## Not so far from the equator

In [7]:
equ_events = mhw_events[np.abs(mhw_events['lat']) < 20.]

In [8]:
equ_events.iloc[np.argsort(equ_events['duration'])[-20:]]

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2018-11-08,0.875,5.625,114,108,737075,737006,2019-01-16
2018-11-07,0.375,5.375,115,99,737075,737005,2019-01-16
2018-11-07,0.625,5.625,115,106,737075,737005,2019-01-16
2018-11-07,0.125,5.375,115,95,737075,737005,2019-01-16
2011-01-26,3.875,-12.125,115,57,734229,734163,2011-04-02
1984-07-13,3.125,-11.875,116,6,724507,724470,1984-08-19
1984-07-15,1.875,-9.625,118,3,724528,724472,1984-09-09
1984-07-14,2.875,-11.875,118,6,724544,724471,1984-09-25
2018-10-16,0.375,4.125,119,93,737065,736983,2019-01-06


In [9]:
ievent = 275128

## Build a Cube

### Cut out near my event

In [10]:
event = mhw_events.iloc[ievent:ievent+1]
event

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2018-10-15,1.625,2.625,131,82,737067,736982,2019-01-08


In [11]:
start_date = event.index.values[0]
start_date

numpy.datetime64('2018-10-15T00:00:00.000000000')

In [12]:
end_date= start_date + pandas.Timedelta(days=event.duration.values[0])
end_date

Timestamp('2019-02-23 00:00:00')

## Just use start for now

### Time

In [13]:
#near_events = mhw_events[start_date:end_date]
neartime_events = mhw_events["2018-10-15":"2019-02-23"]

In [14]:
neartime_events.head()

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2018-12-17,0.125,-68.375,5,87,737048,737045,2018-12-20
2018-12-16,0.125,-68.125,7,81,737048,737044,2018-12-20
2018-12-16,0.125,-67.875,7,84,737048,737044,2018-12-20
2019-02-17,0.125,-67.875,6,85,737109,737107,2019-02-19
2018-12-16,0.125,-67.625,6,90,737045,737044,2018-12-17


### Space

In [15]:
nearst_events = neartime_events[(np.abs(
    neartime_events.lat-event.lat.values[0]) < 10.) & (np.abs(
    neartime_events.lon-event.lon.values[0]) < 10.)]
nearst_events.head()

,lon,lat,duration,ievent,time_peak,time_start,peak_date
date,,,,,,,
2018-10-25,0.125,-7.125,15,78,737004,736992,2018-11-06
2019-02-11,0.125,-7.125,15,79,737105,737101,2019-02-15
2018-10-25,0.125,-6.875,15,79,736998,736992,2018-10-31
2019-02-11,0.125,-6.875,21,80,737105,737101,2019-02-15
2018-10-17,0.125,-6.625,22,80,736997,736984,2018-10-30


### Cube

In [16]:
grid_lon = np.arange(event.lon.values[0]-5., event.lon.values[0]+5.25, 0.25)
grid_lat = np.arange(event.lat.values[0]-5., event.lat.values[0]+5.25, 0.25)

In [17]:
cube = np.zeros((event.duration.values[0], grid_lat.size, grid_lon.size))
cube.shape

(131, 41, 41)

In [18]:
for index, row in nearst_events.iterrows():
    #import pdb; pdb.set_trace()
    ilat = np.argmin(np.abs(grid_lat-row.lat))
    jlon = np.argmin(np.abs(grid_lon-row.lon))
    # Do it!
    t0 = row.time_start-event.time_start.values[0]
    cube[t0:t0+row.duration+1, ilat, jlon] = 1.

In [19]:
np.sum(cube)/cube.size

0.1966523016561389

In [20]:
cube.size

220211

## Viz

In [21]:
x, y, z = np.ogrid[-10:10:20j, -10:10:20j, -10:10:20j]
s = np.sin(x*y*z)/(x*y*z)

In [27]:
mlab.contour3d(s)

### Save

In [141]:
np.save('tst_cube.npy', cube)